<h1>CIRCLE PARALLEL MANIPULATOR INVERSE KINEMATICS<h1>
<h2>INTRODUCTION:<h2> 

<h4>Import Packages :<h4>


In [17]:
import numpy as np

<h4> Robot Parameters <h4>

In [124]:
EndEffectorCoordinates = np.array([[-73.7,19.46,0],
                          [-53.7,54.1,0],
                          [53.7,54.1,0],
                          [73.7,19.46,0],
                          [20,-73.55,0],
                          [-20,-73.55,0]])

BaseCoordinates =  np.array([[-81.63, 0.36, 0],
                        [-41.13 , 70.51, 0],
                        [41.13, 70.51 ,0],
                        [81.63, 0.36, 0],
                        [40.5 ,-70.87 ,0],
                        [-81.63, -70.87, 0]])
    
MotorPlaneAngles = [2.094 , - 2.094 , 5.23, - 5.23, 0, -3.14]; 

MOTOR_COUPLER_LENGTH = 33.81
SS_LINK_LENGTH=146
HOME_POSITION_Z_OFFSET =130.64

In [111]:
def UpdateTranslationVector(delx, dely, delz):
    
    TranslationVector = np.array([[delx], [dely], [delz + HOME_POSITION_Z_OFFSET]]);
    
    #print("Shape of Translation Vector:" + str(TranslationVector.shape))
    return TranslationVector

In [118]:
def UpdateRotationMatrix(Yaw,Pitch,Roll):
    Y = 0.0174*Yaw
    P = 0.0174*Pitch
    R = 0.0174*Roll
    RotationMatrix = np.zeros((3,3))
    
    RotationMatrix[0][0] = np.cos(Y)*np.cos(P)
    RotationMatrix[0][1] = -(np.sin(Y)*np.cos(R))+(np.cos(Y)*np.sin(P)*np.sin(R))
    RotationMatrix[0][2] = (np.sin(Y)*np.sin(R)) + (np.cos(Y)*np.sin(P)*np.cos(R))
    RotationMatrix[1][0] = np.sin(Y)*np.cos(P)
    RotationMatrix[1][1] = (np.cos(Y)*np.cos(R))+(np.sin(Y)*np.sin(P)*np.sin(R))
    RotationMatrix[1][2] = (-np.cos(Y)*np.sin(R))+(np.sin(Y)*np.sin(P)*np.cos(R))
    RotationMatrix[2][0] = -np.sin(P)
    RotationMatrix[2][1] = np.cos(P)*np.sin(R)
    RotationMatrix[2][2] = np.cos(P)*np.cos(R)
    
    #print("Shape of Rotation Matrix:" + str(RotationMatrix.shape))
    return RotationMatrix;
    

In [214]:
def EndEffectorCoordinateTransform(RotationMatrix, TranslationVector):
    ETransform = TranslationVector + np.dot(RotationMatrix,EndEffectorCoordinates.T)
    
    print(ETransform)
    #print("Shape of ETransform:" + str(ETransform.shape))
    return ETransform

In [168]:
def UpdateEffectiveLinkLengths (RotationMatrix,EndEffectorCoordinates,BaseCoordinates,TranslationVector):
    temp = np.dot(RotationMatrix,EndEffectorCoordinates.T)
    LinkVectors = TranslationVector + np.subtract(temp,BaseCoordinates.T)
   
    LinkVectorsSquared = np.multiply(LinkVectors,LinkVectors);
    LinkVectorSquaredSum = np.sum(LinkVectorsSquared, axis = 0, keepdims = True)
    EffectiveLinkLengths = np.sqrt(LinkVectorSquaredSum);
    
    #print("Shape of LinkVectors:" + str(LinkVectors.shape));
    #print("Shape of EffectiveLinkLengths:" + str(EffectiveLinkLengths.shape));
    #print("Shape of LinkVectorsSquared:" + str(LinkVectorsSquared.shape));
    #print("Shape of LinkVectorSquaredSum: "+ str(LinkVectorSquaredSum.shape));
    #print(EffectiveLinkLengths);
    return EffectiveLinkLengths

In [212]:
def UpdateMotorAngles(ETransform,EffectiveLinkLengths):
    
    diffx =  np.subtract(ETransform[0,:], BaseCoordinates[0,:])
    diffy = np.subtract(ETransform[1,:],BaseCoordinates[1,:])
    diffz =  np.subtract(ETransform[2,:], BaseCoordinates[2,:])
    tempM = np.multiply((2*MOTOR_COUPLER_LENGTH),diffz).reshape(1,6)
    tempL = np.square(EffectiveLinkLengths)-(np.square(SS_LINK_LENGTH)-np.square(MOTOR_COUPLER_LENGTH)) 
    temp = np.add(np.multiply(np.cos(MotorPlaneAngles),diffx), np.multiply(np.sin(MotorPlaneAngles), diffy))
    tempN = np.multiply((2*MOTOR_COUPLER_LENGTH),temp).reshape(1,6)
    
    tempsqrt = np.sqrt(np.add(np.square(tempM),np.square(tempN)))
    tempdiv = np.divide(tempL,tempsqrt)
    
    #print("diffx = " + str(diffx))
    #print("diffy = " + str(diffy))
    #print("diffz = " + str(diffz))
   # print(tempL.shape)
   # print(tempM.shape)
    #print(tempN.shape)
    #print(tempdiv.shape)
    print(tempdiv)
    return 0

In [215]:
#Test
delx = 20 
dely = 20 
delz = 0
Yaw = 30  
Pitch = 30  
Roll = 30

TranslationVector = UpdateTranslationVector(delx, dely, delz)
RotationMatrix = UpdateRotationMatrix(Yaw, Pitch, Roll)
EffectiveLinkLengths = UpdateEffectiveLinkLengths(RotationMatrix,EndEffectorCoordinates,BaseCoordinates,TranslationVector)
ETransform = EndEffectorCoordinateTransform(RotationMatrix, TranslationVector)

#print(EffectiveLinkLengths)

[[-39.59401406 -32.07298355  48.62558286  71.15988434  50.96626416
   20.91093216]
 [  5.1802995   44.14655068  90.56602301  68.88821591 -35.73756904
  -53.02601312]
 [175.79873742 180.79823436 127.24700685 102.30291866  88.87857784
  108.82316909]]
